In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
# ids.txt dosyasından device_aids_to_filter listesini oku
with open('ids.txt', 'r') as f:
    device_aids_to_filter = [line.strip() for line in f if line.strip()]

# Filtreleme koşullarını Dask'ın beklediği formatta belirtin
filters = [
        ('device_aid', 'in', device_aids_to_filter)
    ]

In [3]:
df = dd.read_parquet(
        "data/MobilityDataMay2024.paraquet",
        filters=filters ,# Sadece filtreleme uyguluyoruz, tüm sütunlar otomatik dahil olacak
        engine='pyarrow'
    )

In [4]:
df_pd = df.compute()


In [5]:
df_pd


,timestamp,device_aid,latitude,longitude,horizontal_accuracy,altitude,altitude_accuracy,location_method,ip,os,region,district,neighborhood,airport,poi_name,poi_leisure,poi_amenity,grid_id,province
0,1716563371,4d4b939f-3f71-4366-aea0-ebfbb1906b9f,40.987983,29.027896,11.9,70.1,0.0,fused,176.55.74.51,Android,Marmara Bölgesi,Kadıköy,Caferağa Mahallesi,NaN,,,NaN,4100902,Istanbul
1,1716564390,30c55c28-45e8-6980-ad1d-7064b7c84051,40.988467,29.109296,12.0,58.4,0.0,gps,176.54.104.86,Android,Marmara Bölgesi,Ataşehir,Atatürk Mahallesi,NaN,,,NaN,4100910,Istanbul
2,1716563418,d9287169-2268-4958-89dd-192d25dec817,41.023274,29.094867,13.5,188.3,0.0,fused,,Android,Marmara Bölgesi,Ümraniye,Namık Kemal Mahallesi,NaN,,,NaN,4104909,Istanbul
3,1716562879,e55c0589-0589-4539-9783-f96448f259f2,40.994214,29.151767,15.9,160.2,5.0,fused,,Android,Marmara Bölgesi,Ataşehir,Yenişehir Mahallesi,NaN,,,NaN,4101915,Istanbul
4,1716563925,35b845d0-fba5-412a-a8f1-f987fa94ffaa,40.997840,29.027981,7.3,12.6,0.0,gps,176.30.255.159,iOS,Marmara Bölgesi,Kadıköy,Rasimpaşa Mahallesi,NaN,,,NaN,4101902,Istanbul
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8278,1716312267,f3717295-5995-482e-ad4a-58157d2ab66c,41.018621,29.129266,78.9,166.0,10.0,gps,10.63.162.194,iOS,Marmara Bölgesi,Ümraniye,Tepeüstü Mahallesi,NaN,,,NaN,4103912,Istanbul
8279,1716312104,7277915a-3596-4883-8542-041cd3d81804,40.984902,29.037656,13.8,44.7,0.0,fused,,Android,Marmara Bölgesi,Kadıköy,Zühtüpaşa Mahallesi,NaN,,,NaN,4100903,Istanbul
8280,1715412844,2a7e3528-bd56-4f6b-acae-c314bfd970b4,40.990887,29.166584,14.3,0.0,0.0,fused,37.155.7.0,Android,Marmara Bölgesi,Ataşehir,Yeni Çamlıca Mahallesi,NaN,,,NaN,4101916,Istanbul
8281,1716311875,02712699-4a57-400b-a04a-d39bd5c8eba1,40.971018,29.117447,12.0,122.8,0.0,fused,176.234.208.47,Android,Marmara Bölgesi,Ataşehir,İçerenköy Mahallesi,NaN,,,NaN,4099911,Istanbul


In [6]:
df_pd["datetime"] = pd.to_datetime(df_pd["timestamp"], unit='s')

In [7]:
def zaman_bolumu(saat):
    if 6 <= saat < 12:
        return "sabah"
    elif 12 <= saat < 18:
        return "oglen"
    elif 18 <= saat < 24:
        return "aksam"
    else:
        return "gece"

df_pd["zaman_bolumu"] = df_pd["datetime"].dt.hour.apply(zaman_bolumu)


In [8]:
df_pd

,timestamp,device_aid,latitude,longitude,horizontal_accuracy,altitude,altitude_accuracy,location_method,ip,os,...,district,neighborhood,airport,poi_name,poi_leisure,poi_amenity,grid_id,province,datetime,zaman_bolumu
0,1716563371,4d4b939f-3f71-4366-aea0-ebfbb1906b9f,40.987983,29.027896,11.9,70.1,0.0,fused,176.55.74.51,Android,...,Kadıköy,Caferağa Mahallesi,NaN,,,NaN,4100902,Istanbul,2024-05-24 15:09:31,oglen
1,1716564390,30c55c28-45e8-6980-ad1d-7064b7c84051,40.988467,29.109296,12.0,58.4,0.0,gps,176.54.104.86,Android,...,Ataşehir,Atatürk Mahallesi,NaN,,,NaN,4100910,Istanbul,2024-05-24 15:26:30,oglen
2,1716563418,d9287169-2268-4958-89dd-192d25dec817,41.023274,29.094867,13.5,188.3,0.0,fused,,Android,...,Ümraniye,Namık Kemal Mahallesi,NaN,,,NaN,4104909,Istanbul,2024-05-24 15:10:18,oglen
3,1716562879,e55c0589-0589-4539-9783-f96448f259f2,40.994214,29.151767,15.9,160.2,5.0,fused,,Android,...,Ataşehir,Yenişehir Mahallesi,NaN,,,NaN,4101915,Istanbul,2024-05-24 15:01:19,oglen
4,1716563925,35b845d0-fba5-412a-a8f1-f987fa94ffaa,40.997840,29.027981,7.3,12.6,0.0,gps,176.30.255.159,iOS,...,Kadıköy,Rasimpaşa Mahallesi,NaN,,,NaN,4101902,Istanbul,2024-05-24 15:18:45,oglen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8278,1716312267,f3717295-5995-482e-ad4a-58157d2ab66c,41.018621,29.129266,78.9,166.0,10.0,gps,10.63.162.194,iOS,...,Ümraniye,Tepeüstü Mahallesi,NaN,,,NaN,4103912,Istanbul,2024-05-21 17:24:27,oglen
8279,1716312104,7277915a-3596-4883-8542-041cd3d81804,40.984902,29.037656,13.8,44.7,0.0,fused,,Android,...,Kadıköy,Zühtüpaşa Mahallesi,NaN,,,NaN,4100903,Istanbul,2024-05-21 17:21:44,oglen
8280,1715412844,2a7e3528-bd56-4f6b-acae-c314bfd970b4,40.990887,29.166584,14.3,0.0,0.0,fused,37.155.7.0,Android,...,Ataşehir,Yeni Çamlıca Mahallesi,NaN,,,NaN,4101916,Istanbul,2024-05-11 07:34:04,sabah
8281,1716311875,02712699-4a57-400b-a04a-d39bd5c8eba1,40.971018,29.117447,12.0,122.8,0.0,fused,176.234.208.47,Android,...,Ataşehir,İçerenköy Mahallesi,NaN,,,NaN,4099911,Istanbul,2024-05-21 17:17:55,oglen


In [9]:
df_pd=df_pd.drop(columns=["airport"],axis=1)
df_pd=df_pd.drop(columns=["poi_leisure"],axis=1)
df_pd=df_pd.drop(columns=["poi_amenity"],axis=1)
df_pd=df_pd.drop(columns=["poi_name"],axis=1)
df_pd=df_pd.drop(columns=["altitude"],axis=1)
df_pd=df_pd.drop(columns=["altitude_accuracy"],axis=1)
df_pd=df_pd.drop(columns=["location_method"],axis=1)
df_pd=df_pd.drop(columns=["ip"],axis=1)
df_pd=df_pd.drop(columns=["os"],axis=1)
df_pd=df_pd.drop(columns=["timestamp"],axis=1)
df_pd=df_pd.drop(columns=["region"],axis=1)
df_pd=df_pd.drop(columns=["province"],axis=1)

In [10]:
df_pd

,device_aid,latitude,longitude,horizontal_accuracy,district,neighborhood,grid_id,datetime,zaman_bolumu
0,4d4b939f-3f71-4366-aea0-ebfbb1906b9f,40.987983,29.027896,11.9,Kadıköy,Caferağa Mahallesi,4100902,2024-05-24 15:09:31,oglen
1,30c55c28-45e8-6980-ad1d-7064b7c84051,40.988467,29.109296,12.0,Ataşehir,Atatürk Mahallesi,4100910,2024-05-24 15:26:30,oglen
2,d9287169-2268-4958-89dd-192d25dec817,41.023274,29.094867,13.5,Ümraniye,Namık Kemal Mahallesi,4104909,2024-05-24 15:10:18,oglen
3,e55c0589-0589-4539-9783-f96448f259f2,40.994214,29.151767,15.9,Ataşehir,Yenişehir Mahallesi,4101915,2024-05-24 15:01:19,oglen
4,35b845d0-fba5-412a-a8f1-f987fa94ffaa,40.997840,29.027981,7.3,Kadıköy,Rasimpaşa Mahallesi,4101902,2024-05-24 15:18:45,oglen
...,...,...,...,...,...,...,...,...,...
8278,f3717295-5995-482e-ad4a-58157d2ab66c,41.018621,29.129266,78.9,Ümraniye,Tepeüstü Mahallesi,4103912,2024-05-21 17:24:27,oglen
8279,7277915a-3596-4883-8542-041cd3d81804,40.984902,29.037656,13.8,Kadıköy,Zühtüpaşa Mahallesi,4100903,2024-05-21 17:21:44,oglen
8280,2a7e3528-bd56-4f6b-acae-c314bfd970b4,40.990887,29.166584,14.3,Ataşehir,Yeni Çamlıca Mahallesi,4101916,2024-05-11 07:34:04,sabah
8281,02712699-4a57-400b-a04a-d39bd5c8eba1,40.971018,29.117447,12.0,Ataşehir,İçerenköy Mahallesi,4099911,2024-05-21 17:17:55,oglen


In [11]:
df_pd = df_pd[df_pd["horizontal_accuracy"]<=200]

In [12]:
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np


In [13]:
df_pd["date"] = df_pd["datetime"].dt.date

data = (
    df_pd.groupby(["device_aid", "grid_id", "date", "zaman_bolumu"])
    .first()
    .reset_index()
)

data

/tmp/ipykernel_213939/1521950039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pd["date"] = df_pd["datetime"].dt.date


,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55
...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07


In [14]:
df_restoran = pd.read_csv("Clustered_Restaurants.csv")
df_restoran = df_restoran.dropna()

# Konumları radyana çevir
mob_coords = np.radians(data[["latitude", "longitude"]])
res_coords = np.radians(df_restoran[["Enlem", "Boylam"]])

# BallTree oluştur ve Haversine metriğini kullan
tree = BallTree(res_coords, metric='haversine')

# En yakın 1 restoranı bul, sonuç: (mesafe, indeks)
dist, ind = tree.query(mob_coords, k=1)

# Haversine mesafesi radian cinsinden, bunu metreye çevirelim:
# Dünya'nın yarıçapı yaklaşık 6371 km
dist_meters = dist.flatten() * 6371000

# Bu noktada horizontal_accuracy değerini alıyoruz
accuracy_thresholds = data["horizontal_accuracy"].values

# Her satır için mesafe < horizontal_accuracy ise eşleşme var
mask = dist_meters < accuracy_thresholds

# Yeni kolonlar
data["matched_restaurant"] = None
data["restaurant_cluster"] = None
data["distance_to_restaurant"] = dist_meters

# Eşleşenleri yaz
matched_indices = ind.flatten()

data.loc[mask, "matched_restaurant"] = df_restoran.iloc[matched_indices[mask]]["Restoran Çeşidi"].values
data.loc[mask, "restaurant_cluster"] = df_restoran.iloc[matched_indices[mask]]["Cluster"].values


In [15]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,matched_restaurant,restaurant_cluster,distance_to_restaurant
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,Modern Pub & Bistro,Zengin Restoran,41.444034
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,None,None,24.510692
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,Pub & Bistro & Kokteyl Bar,Zengin Restoran,19.049424
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,None,None,406.554943
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,Modern Pub & Bistro,Orta Halli Restoran,56.320458
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,None,None,1159.031044
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,None,None,1208.849886
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,None,None,749.735787
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,None,None,892.135337


In [16]:
df_veteriner = pd.read_csv("Clustered_Veteriner.csv")
df_veteriner = df_veteriner.dropna()

# Konumları radyana çevir
mob_coords = np.radians(data[["latitude", "longitude"]])
vet_coords = np.radians(df_veteriner[["lat", "lng"]])

# BallTree oluştur ve Haversine metriğini kullan
tree = BallTree(vet_coords, metric='haversine')

# En yakın 1 restoranı bul, sonuç: (mesafe, indeks)
dist, ind = tree.query(mob_coords, k=1)

# Haversine mesafesi radian cinsinden, bunu metreye çevirelim:
# Dünya'nın yarıçapı yaklaşık 6371 km
dist_meters = dist.flatten() * 6371000

# Bu noktada horizontal_accuracy değerini alıyoruz
accuracy_thresholds = data["horizontal_accuracy"].values

# Her satır için mesafe < horizontal_accuracy ise eşleşme var
mask = dist_meters < accuracy_thresholds

# Yeni kolonlar
data["matched_veteriner"] = None
data["veteriner_cluster"] = None
data["distance_to_veteriner"] = dist_meters

# Eşleşenleri yaz
matched_indices = ind.flatten()

data.loc[mask, "matched_veteriner"] = df_veteriner.iloc[matched_indices[mask]]["Tipi"].values
data.loc[mask, "veteriner_cluster"] = df_veteriner.iloc[matched_indices[mask]]["Cluster"].values

In [17]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,matched_restaurant,restaurant_cluster,distance_to_restaurant,matched_veteriner,veteriner_cluster,distance_to_veteriner
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,Modern Pub & Bistro,Zengin Restoran,41.444034,None,None,199.715047
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,None,None,24.510692,None,None,95.623052
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,Pub & Bistro & Kokteyl Bar,Zengin Restoran,19.049424,None,None,268.466776
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,None,None,406.554943,None,None,263.120625
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,Modern Pub & Bistro,Orta Halli Restoran,56.320458,None,None,113.148069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,None,None,1159.031044,None,None,265.208381
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,None,None,1208.849886,None,None,373.531149
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,None,None,749.735787,None,None,269.646750
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,None,None,892.135337,None,None,421.246825


In [18]:
df_kahve = pd.read_csv("Kahve_New.csv")
df_kahve = df_kahve.dropna()

# Konumları radyana çevir
mob_coords = np.radians(data[["latitude", "longitude"]])
cof_coords = np.radians(df_kahve[["latitude", "longitude"]])

# BallTree oluştur ve Haversine metriğini kullan
tree = BallTree(cof_coords, metric='haversine')

# En yakın 1 restoranı bul, sonuç: (mesafe, indeks)
dist, ind = tree.query(mob_coords, k=1)

# Haversine mesafesi radian cinsinden, bunu metreye çevirelim:
# Dünya'nın yarıçapı yaklaşık 6371 km
dist_meters = dist.flatten() * 6371000

# Bu noktada horizontal_accuracy değerini alıyoruz
accuracy_thresholds = data["horizontal_accuracy"].values

# Her satır için mesafe < horizontal_accuracy ise eşleşme var
mask = dist_meters < accuracy_thresholds

# Yeni kolonlar
data["matched_coffee_name"] = None
data["distance_to_coffee"] = dist_meters

# Eşleşenleri yaz
matched_indices = ind.flatten()

data.loc[mask, "matched_coffee_name"] = df_kahve.iloc[matched_indices[mask]]["isim"].values

In [19]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,matched_restaurant,restaurant_cluster,distance_to_restaurant,matched_veteriner,veteriner_cluster,distance_to_veteriner,matched_coffee_name,distance_to_coffee
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,Modern Pub & Bistro,Zengin Restoran,41.444034,None,None,199.715047,BigChefs Suadiye,28.889632
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,None,None,24.510692,None,None,95.623052,None,99.111750
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,Pub & Bistro & Kokteyl Bar,Zengin Restoran,19.049424,None,None,268.466776,Kahve Dünyası - Bağdat Caddesi Beyaz Köşk,14.347108
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,None,None,406.554943,None,None,263.120625,None,75.858481
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,Modern Pub & Bistro,Orta Halli Restoran,56.320458,None,None,113.148069,Caribou Coffee,34.369631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,None,None,1159.031044,None,None,265.208381,None,1136.700967
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,None,None,1208.849886,None,None,373.531149,None,1261.411049
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,None,None,749.735787,None,None,269.646750,None,646.919497
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,None,None,892.135337,None,None,421.246825,None,824.851825


In [20]:
# CHECKPOINT

In [21]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
import json
import warnings

# Uyarıları kapatmak isterseniz
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)

# 1) Veri setinizi yükleyin
# data = pd.read_csv('your_data_file.csv')

# 2) Orijinal indeks bilgisini tutalım
data['original_index'] = data.index

# 3) Cihaz lokasyonlarını GeoDataFrame'e dönüştürelim
gdf_data = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data.longitude, data.latitude),
    crs="EPSG:4326"
)

# 4) POI geojson'unuzu okuyup GeoDataFrame'e dönüştürelim
poi_file_path = 'data/Polygons/poi.geojson'  # kendi yolunuza göre düzenleyin
with open(poi_file_path, 'r', encoding='utf-8') as f:
    poi_json = json.load(f)

poi_features = []
for feat in poi_json['features']:
    props = feat['properties']
    geom_type = feat['geometry']['type']
    coords = feat['geometry']['coordinates']
    try:
        if geom_type == "MultiPolygon":
            polys = []
            for poly_coords in coords:
                exterior = poly_coords[0]
                interiors = poly_coords[1:]
                polys.append(Polygon(exterior, interiors))
            geometry = MultiPolygon(polys)
        elif geom_type == "Polygon":
            exterior = coords[0]
            interiors = coords[1:]
            geometry = Polygon(exterior, interiors)
        else:
            # Point/LineString vs. Polygon ayrımı gerekiyorsa ekleyebilirsiniz
            continue
    except Exception:
        continue

    poi_features.append({
        'geometry': geometry,
        **{k: props.get(k) for k in ['NAME','NAME_EN','AMENITY','LEISURE','SHOP','TOURISM','SPORT','OFFICE','MAN_MADE','OSM_TYPE','OSM_ID']}
    })

gdf_poi = gpd.GeoDataFrame(poi_features, crs="EPSG:4326")

# 5) Her iki GeoDataFrame'i metre cinsine uygun projeksiyona alalım
gdf_data_proj = gdf_data.to_crs("EPSG:3857")
gdf_poi_proj  = gdf_poi.to_crs("EPSG:3857")

# 6) Noktalardan kendi horizontal_accuracy değerine göre buffer oluşturup kesişme kontrolü yapalım
gdf_data_proj['buffered'] = gdf_data_proj.geometry.buffer(gdf_data_proj['horizontal_accuracy'])

joined = gpd.sjoin(
    gdf_data_proj.set_geometry('buffered'),
    gdf_poi_proj,
    how="left",
    predicate="intersects"
)

# 7) Her orijinal sinyal için eşleşen POI isimlerini listele
import numpy as np

matched = (
    joined
    .groupby('original_index')['NAME']
    .apply(lambda names: names.dropna().tolist())
    .to_frame(name='matched_pois')
)

# 8) Sonuçları orijinal tabloya geri birleştir
data = (
    data
    .merge(matched, left_on='original_index', right_index=True, how='left')
    .drop(columns=['original_index'])
)

# 9) Boş listeleri None yapalım
data['matched_pois'] = data['matched_pois'].apply(lambda lst: lst if lst else None)

print("İşlem tamam! Her nokta için kendi horizontal_accuracy değerine göre POI eşleşmeleri bulundu.")


İşlem tamam! Her nokta için kendi horizontal_accuracy değerine göre POI eşleşmeleri bulundu.


In [22]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,matched_restaurant,restaurant_cluster,distance_to_restaurant,matched_veteriner,veteriner_cluster,distance_to_veteriner,matched_coffee_name,distance_to_coffee,matched_pois
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,Modern Pub & Bistro,Zengin Restoran,41.444034,None,None,199.715047,BigChefs Suadiye,28.889632,None
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,None,None,24.510692,None,None,95.623052,None,99.111750,None
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,Pub & Bistro & Kokteyl Bar,Zengin Restoran,19.049424,None,None,268.466776,Kahve Dünyası - Bağdat Caddesi Beyaz Köşk,14.347108,None
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,None,None,406.554943,None,None,263.120625,None,75.858481,[İspark İçerenköy Açık Otoparkı]
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,Modern Pub & Bistro,Orta Halli Restoran,56.320458,None,None,113.148069,Caribou Coffee,34.369631,[Palladium Alışveriş Merkezi]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,None,None,1159.031044,None,None,265.208381,None,1136.700967,"[Clubsporium Halı Saha, Clubsporium Halı Saha]"
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,None,None,1208.849886,None,None,373.531149,None,1261.411049,None
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,None,None,749.735787,None,None,269.646750,None,646.919497,[Tatar Ağası Camii]
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,None,None,892.135337,None,None,421.246825,None,824.851825,None


In [23]:
import json
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

# 4) POI geojson'unu okuyup GeoDataFrame'e dönüştürme
poi_file_path = 'data/Polygons/merged_polygons_p_schools.geojson'

with open(poi_file_path, 'r', encoding='utf-8') as f:
    poi_json = json.load(f)

poi_features = []
for feat in poi_json['features']:
    props = feat['properties']
    geom_type = feat['geometry']['type']
    coords = feat['geometry']['coordinates']
    
    try:
        if geom_type == "MultiPolygon":
            polys = []
            for poly_group in coords:
                for poly_coords in poly_group:
                    exterior = poly_coords[0]
                    interiors = poly_coords[1:]
                    polys.append(Polygon(exterior, interiors))
            geometry = MultiPolygon(polys)
        elif geom_type == "Polygon":
            exterior = coords[0]
            interiors = coords[1:]
            geometry = Polygon(exterior, interiors)
        else:
            continue
    except Exception as e:
        print(f"Geometry parsing error: {e}")
        continue

    poi_features.append({
        'geometry': geometry,
        'name': props.get('name'),
        'kategori': props.get('kategori')
    })

gdf_poi = gpd.GeoDataFrame(poi_features, crs="EPSG:4326")

# 5) Projeksiyonları metre bazlına çevirme
gdf_data_proj = gdf_data.to_crs("EPSG:3857")
gdf_poi_proj = gdf_poi.to_crs("EPSG:3857")

# 6) Buffer oluşturma ve kesişim kontrolü
gdf_data_proj['buffered'] = gdf_data_proj.geometry.buffer(gdf_data_proj['horizontal_accuracy'])

# 7) Mekansal birleştirme
joined = gpd.sjoin(
    gdf_data_proj.set_geometry('buffered'),
    gdf_poi_proj,
    how="left",
    predicate="intersects"
)

# 7.5) Orijinal indeks bilgisini tutalım
data['original_index'] = data.index

# 8) Eşleşen POI'ları gruplama ve sadece isimleri birleştirerek stringe çevirme
matched = (
    joined.groupby('original_index')
    .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))
    .to_frame(name='matched_schools')
)

# 9) Orijinal veriyle birleştirme
data = (
    data
    .merge(matched, left_on='original_index', right_index=True, how='left')
    .drop(columns=['original_index'])
)

# 10) Boş değerleri None yapma
# Eğer matched_pois boş string ise None olarak ayarlayalım
data['matched_schools'] = data['matched_schools'].replace('', None)

print("İşlem başarıyla tamamlandı! Sonuçlar:")

İşlem başarıyla tamamlandı! Sonuçlar:


/tmp/ipykernel_213939/328395034.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))


In [24]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,matched_restaurant,restaurant_cluster,distance_to_restaurant,matched_veteriner,veteriner_cluster,distance_to_veteriner,matched_coffee_name,distance_to_coffee,matched_pois,matched_schools
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,Modern Pub & Bistro,Zengin Restoran,41.444034,None,None,199.715047,BigChefs Suadiye,28.889632,None,None
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,None,None,24.510692,None,None,95.623052,None,99.111750,None,None
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,Pub & Bistro & Kokteyl Bar,Zengin Restoran,19.049424,None,None,268.466776,Kahve Dünyası - Bağdat Caddesi Beyaz Köşk,14.347108,None,None
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,None,None,406.554943,None,None,263.120625,None,75.858481,[İspark İçerenköy Açık Otoparkı],None
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,Modern Pub & Bistro,Orta Halli Restoran,56.320458,None,None,113.148069,Caribou Coffee,34.369631,[Palladium Alışveriş Merkezi],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,None,None,1159.031044,None,None,265.208381,None,1136.700967,"[Clubsporium Halı Saha, Clubsporium Halı Saha]",None
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,None,None,1208.849886,None,None,373.531149,None,1261.411049,None,None
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,None,None,749.735787,None,None,269.646750,None,646.919497,[Tatar Ağası Camii],None
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,None,None,892.135337,None,None,421.246825,None,824.851825,None,None


In [25]:
import json
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

# 4) POI geojson'unu okuyup GeoDataFrame'e dönüştürme
poi_file_path = 'data/Polygons/merged_polygons_hotels.geojson'

with open(poi_file_path, 'r', encoding='utf-8') as f:
    poi_json = json.load(f)

poi_features = []
for feat in poi_json['features']:
    props = feat['properties']
    geom_type = feat['geometry']['type']
    coords = feat['geometry']['coordinates']
    
    try:
        if geom_type == "MultiPolygon":
            polys = []
            for poly_group in coords:
                for poly_coords in poly_group:
                    exterior = poly_coords[0]
                    interiors = poly_coords[1:]
                    polys.append(Polygon(exterior, interiors))
            geometry = MultiPolygon(polys)
        elif geom_type == "Polygon":
            exterior = coords[0]
            interiors = coords[1:]
            geometry = Polygon(exterior, interiors)
        else:
            continue
    except Exception as e:
        print(f"Geometry parsing error: {e}")
        continue

    poi_features.append({
        'geometry': geometry,
        'name': props.get('MusteriTabelaAdi'),
        
    })

gdf_poi = gpd.GeoDataFrame(poi_features, crs="EPSG:4326")

# 5) Projeksiyonları metre bazlına çevirme
gdf_data_proj = gdf_data.to_crs("EPSG:3857")
gdf_poi_proj = gdf_poi.to_crs("EPSG:3857")

# 6) Buffer oluşturma ve kesişim kontrolü
gdf_data_proj['buffered'] = gdf_data_proj.geometry.buffer(gdf_data_proj['horizontal_accuracy'])

# 7) Mekansal birleştirme
joined = gpd.sjoin(
    gdf_data_proj.set_geometry('buffered'),
    gdf_poi_proj,
    how="left",
    predicate="intersects"
)

# 7.5) Orijinal indeks bilgisini tutalım
data['original_index'] = data.index

# 8) Eşleşen POI'ları gruplama ve sadece isimleri birleştirerek stringe çevirme
matched = (
    joined.groupby('original_index')
    .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))
    .to_frame(name='matched_hotels')
)

# 9) Orijinal veriyle birleştirme
data = (
    data
    .merge(matched, left_on='original_index', right_index=True, how='left')
    .drop(columns=['original_index'])
)

# 10) Boş değerleri None yapma
# Eğer matched_pois boş string ise None olarak ayarlayalım
data['matched_hotels'] = data['matched_hotels'].replace('', None)

print("İşlem başarıyla tamamlandı! Sonuçlar:")

İşlem başarıyla tamamlandı! Sonuçlar:


/tmp/ipykernel_213939/740250501.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))


In [26]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,...,restaurant_cluster,distance_to_restaurant,matched_veteriner,veteriner_cluster,distance_to_veteriner,matched_coffee_name,distance_to_coffee,matched_pois,matched_schools,matched_hotels
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,...,Zengin Restoran,41.444034,None,None,199.715047,BigChefs Suadiye,28.889632,None,None,None
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,...,None,24.510692,None,None,95.623052,None,99.111750,None,None,None
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,...,Zengin Restoran,19.049424,None,None,268.466776,Kahve Dünyası - Bağdat Caddesi Beyaz Köşk,14.347108,None,None,None
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,...,None,406.554943,None,None,263.120625,None,75.858481,[İspark İçerenköy Açık Otoparkı],None,None
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,...,Orta Halli Restoran,56.320458,None,None,113.148069,Caribou Coffee,34.369631,[Palladium Alışveriş Merkezi],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,...,None,1159.031044,None,None,265.208381,None,1136.700967,"[Clubsporium Halı Saha, Clubsporium Halı Saha]",None,None
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,...,None,1208.849886,None,None,373.531149,None,1261.411049,None,None,None
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,...,None,749.735787,None,None,269.646750,None,646.919497,[Tatar Ağası Camii],None,None
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,...,None,892.135337,None,None,421.246825,None,824.851825,None,None,None


In [27]:
import json
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

# 4) POI geojson'unu okuyup GeoDataFrame'e dönüştürme
poi_file_path = 'data/Polygons/merged_polygons_turkey_sites.geojson'

with open(poi_file_path, 'r', encoding='utf-8') as f:
    poi_json = json.load(f)

poi_features = []
for feat in poi_json['features']:
    props = feat['properties']
    geom_type = feat['geometry']['type']
    coords = feat['geometry']['coordinates']
    
    try:
        if geom_type == "MultiPolygon":
            polys = []
            for poly_group in coords:
                for poly_coords in poly_group:
                    exterior = poly_coords[0]
                    interiors = poly_coords[1:]
                    polys.append(Polygon(exterior, interiors))
            geometry = MultiPolygon(polys)
        elif geom_type == "Polygon":
            exterior = coords[0]
            interiors = coords[1:]
            geometry = Polygon(exterior, interiors)
        else:
            continue
    except Exception as e:
        print(f"Geometry parsing error: {e}")
        continue

    poi_features.append({
        'geometry': geometry,
        'name': props.get('Konut Siteleri'),
        
    })

gdf_poi = gpd.GeoDataFrame(poi_features, crs="EPSG:4326")

# 5) Projeksiyonları metre bazlına çevirme
gdf_data_proj = gdf_data.to_crs("EPSG:3857")
gdf_poi_proj = gdf_poi.to_crs("EPSG:3857")

# 6) Buffer oluşturma ve kesişim kontrolü
gdf_data_proj['buffered'] = gdf_data_proj.geometry.buffer(gdf_data_proj['horizontal_accuracy'])

# 7) Mekansal birleştirme
joined = gpd.sjoin(
    gdf_data_proj.set_geometry('buffered'),
    gdf_poi_proj,
    how="left",
    predicate="intersects"
)

# 7.5) Orijinal indeks bilgisini tutalım
data['original_index'] = data.index

# 8) Eşleşen POI'ları gruplama ve sadece isimleri birleştirerek stringe çevirme
matched = (
    joined.groupby('original_index')
    .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))
    .to_frame(name='matched_turkey_sites')
)

# 9) Orijinal veriyle birleştirme
data = (
    data
    .merge(matched, left_on='original_index', right_index=True, how='left')
    .drop(columns=['original_index'])
)

# 10) Boş değerleri None yapma
# Eğer matched_pois boş string ise None olarak ayarlayalım
data['matched_turkey_sites'] = data['matched_turkey_sites'].replace('', None)

print("İşlem başarıyla tamamlandı! Sonuçlar:")

İşlem başarıyla tamamlandı! Sonuçlar:


/tmp/ipykernel_213939/1761967936.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))


In [28]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,...,distance_to_restaurant,matched_veteriner,veteriner_cluster,distance_to_veteriner,matched_coffee_name,distance_to_coffee,matched_pois,matched_schools,matched_hotels,matched_turkey_sites
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,...,41.444034,None,None,199.715047,BigChefs Suadiye,28.889632,None,None,None,None
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,...,24.510692,None,None,95.623052,None,99.111750,None,None,None,None
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,...,19.049424,None,None,268.466776,Kahve Dünyası - Bağdat Caddesi Beyaz Köşk,14.347108,None,None,None,None
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,...,406.554943,None,None,263.120625,None,75.858481,[İspark İçerenköy Açık Otoparkı],None,None,None
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,...,56.320458,None,None,113.148069,Caribou Coffee,34.369631,[Palladium Alışveriş Merkezi],None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,...,1159.031044,None,None,265.208381,None,1136.700967,"[Clubsporium Halı Saha, Clubsporium Halı Saha]",None,None,None
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,...,1208.849886,None,None,373.531149,None,1261.411049,None,None,None,None
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,...,749.735787,None,None,269.646750,None,646.919497,[Tatar Ağası Camii],None,None,None
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,...,892.135337,None,None,421.246825,None,824.851825,None,None,None,None


In [29]:
import json
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon

# 4) POI geojson'unu okuyup GeoDataFrame'e dönüştürme
poi_file_path = 'data/Polygons/merged_polygons_luxury_houses.geojson'

with open(poi_file_path, 'r', encoding='utf-8') as f:
    poi_json = json.load(f)

poi_features = []
for feat in poi_json['features']:
    props = feat['properties']
    geom_type = feat['geometry']['type']
    coords = feat['geometry']['coordinates']
    
    try:
        if geom_type == "MultiPolygon":
            polys = []
            for poly_group in coords:
                for poly_coords in poly_group:
                    exterior = poly_coords[0]
                    interiors = poly_coords[1:]
                    polys.append(Polygon(exterior, interiors))
            geometry = MultiPolygon(polys)
        elif geom_type == "Polygon":
            exterior = coords[0]
            interiors = coords[1:]
            geometry = Polygon(exterior, interiors)
        else:
            continue
    except Exception as e:
        print(f"Geometry parsing error: {e}")
        continue

    poi_features.append({
        'geometry': geometry,
        'name': props.get('POI Adı'),
        'kategori': props.get('kategori')
        
    })

gdf_poi = gpd.GeoDataFrame(poi_features, crs="EPSG:4326")

# 5) Projeksiyonları metre bazlına çevirme
gdf_data_proj = gdf_data.to_crs("EPSG:3857")
gdf_poi_proj = gdf_poi.to_crs("EPSG:3857")

# 6) Buffer oluşturma ve kesişim kontrolü
gdf_data_proj['buffered'] = gdf_data_proj.geometry.buffer(gdf_data_proj['horizontal_accuracy'])

# 7) Mekansal birleştirme
joined = gpd.sjoin(
    gdf_data_proj.set_geometry('buffered'),
    gdf_poi_proj,
    how="left",
    predicate="intersects"
)

# 7.5) Orijinal indeks bilgisini tutalım
data['original_index'] = data.index

# 8) Eşleşen POI'ları gruplama ve sadece isimleri birleştirerek stringe çevirme
matched = (
    joined.groupby('original_index')
    .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))
    .to_frame(name='matched_turkey_luxury_houses')
)

# 9) Orijinal veriyle birleştirme
data = (
    data
    .merge(matched, left_on='original_index', right_index=True, how='left')
    .drop(columns=['original_index'])
)

# 10) Boş değerleri None yapma
# Eğer matched_pois boş string ise None olarak ayarlayalım
data['matched_turkey_luxury_houses'] = data['matched_turkey_luxury_houses'].replace('', None)

print("İşlem başarıyla tamamlandı! Sonuçlar:")

İşlem başarıyla tamamlandı! Sonuçlar:


/tmp/ipykernel_213939/1682703945.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: ', '.join(grp['name'].dropna().unique()))


In [30]:
data

,device_aid,grid_id,date,zaman_bolumu,latitude,longitude,horizontal_accuracy,district,neighborhood,datetime,...,matched_veteriner,veteriner_cluster,distance_to_veteriner,matched_coffee_name,distance_to_coffee,matched_pois,matched_schools,matched_hotels,matched_turkey_sites,matched_turkey_luxury_houses
0,01069bba-c0ed-4999-8515-22754c3fee3e,4097908,2024-05-24,sabah,40.956284,29.089339,65.0,Kadıköy,Bostancı Mahallesi,2024-05-24 08:20:04,...,None,None,199.715047,BigChefs Suadiye,28.889632,None,None,None,None,None
1,01069bba-c0ed-4999-8515-22754c3fee3e,4098907,2024-05-24,sabah,40.963128,29.075477,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:36:02,...,None,None,95.623052,None,99.111750,None,None,None,None,None
2,01069bba-c0ed-4999-8515-22754c3fee3e,4098908,2024-05-24,sabah,40.960540,29.080104,20.0,Kadıköy,Suadiye Mahallesi,2024-05-24 08:25:59,...,None,None,268.466776,Kahve Dünyası - Bağdat Caddesi Beyaz Köşk,14.347108,None,None,None,None,None
3,01069bba-c0ed-4999-8515-22754c3fee3e,4099910,2024-05-24,sabah,40.976616,29.103997,65.0,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:34:39,...,None,None,263.120625,None,75.858481,[İspark İçerenköy Açık Otoparkı],None,None,None,None
4,01069bba-c0ed-4999-8515-22754c3fee3e,4100909,2024-05-24,sabah,40.986200,29.099270,65.0,Ataşehir,Barbaros Mahallesi,2024-05-24 08:09:55,...,None,None,113.148069,Caribou Coffee,34.369631,[Palladium Alışveriş Merkezi],None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,oglen,40.959350,29.110321,64.0,Kadıköy,Bostancı Mahallesi,2024-05-24 15:50:43,...,None,None,265.208381,None,1136.700967,"[Clubsporium Halı Saha, Clubsporium Halı Saha]",None,None,None,None
20683,fa697609-b068-4af2-8d7d-c3cc53fb128a,4097911,2024-05-24,sabah,40.959670,29.111763,67.6,Ataşehir,İçerenköy Mahallesi,2024-05-24 09:31:11,...,None,None,373.531149,None,1261.411049,None,None,None,None,None
20684,fa697609-b068-4af2-8d7d-c3cc53fb128a,4098910,2024-05-23,oglen,40.960964,29.102520,65.0,Kadıköy,Bostancı Mahallesi,2024-05-23 16:38:18,...,None,None,269.646750,None,646.919497,[Tatar Ağası Camii],None,None,None,None
20685,fa697609-b068-4af2-8d7d-c3cc53fb128a,4102914,2024-05-29,aksam,41.007425,29.147105,39.7,Ümraniye,Yukarı Dudullu Mahallesi,2024-05-29 22:04:07,...,None,None,421.246825,None,824.851825,None,None,None,None,None


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20687 entries, 0 to 20686
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   device_aid                    20687 non-null  string        
 1   grid_id                       20687 non-null  int64         
 2   date                          20687 non-null  object        
 3   zaman_bolumu                  20687 non-null  object        
 4   latitude                      20687 non-null  float64       
 5   longitude                     20687 non-null  float64       
 6   horizontal_accuracy           20687 non-null  float64       
 7   district                      20687 non-null  string        
 8   neighborhood                  20687 non-null  string        
 9   datetime                      20687 non-null  datetime64[ns]
 10  matched_restaurant            235 non-null    object        
 11  restaurant_cluster          

In [32]:
import pandas as pd

# 1. Sadece "gece" zaman dilimini filtrele
gece_df = data[data['zaman_bolumu'] == 'gece']

# 2. Kişi ve mahalle bazında gece saatlerinde kaç kez bulunmuş, say
gece_counts = gece_df.groupby(['device_aid', 'neighborhood']).size().reset_index(name='gece_sayisi')

# 3. Her kişi için en fazla bulunduğu mahalleyi bul
en_cok_gece_mahallesi = gece_counts.sort_values('gece_sayisi', ascending=False).drop_duplicates('device_aid')
en_cok_gece_mahallesi = en_cok_gece_mahallesi.reset_index(drop=True)



# 4. Sonuçları tablo olarak göster
print(en_cok_gece_mahallesi)


                              device_aid          neighborhood  gece_sayisi
0   b0cc71d0-d5a1-4e06-941c-1a75a50ef415     İnkılap Mahallesi           56
1   01069bba-c0ed-4999-8515-22754c3fee3e     Atatürk Mahallesi           49
2   77a8c940-cabd-486d-b32d-eb098340d456      Çamlık Mahallesi           48
3   e90971b7-e7e4-4b32-a8be-3378700412a4   Kozyatağı Mahallesi           43
4   29d09d1a-6346-4c36-b1b1-e77fb8e9afe8    Esenkent Mahallesi           41
..                                   ...                   ...          ...
85  16e1ec7f-01be-685d-21ef-0091fa1d7e3a       Örnek Mahallesi            3
86  e71ec75b-2859-47a3-96cd-37559392022c   Kozyatağı Mahallesi            2
87  992f4683-7fbf-40a2-8bf9-e63da6682681    Şerifali Mahallesi            1
88  e21972df-f04d-4d23-b785-20a3305aca96    Barbaros Mahallesi            1
89  fa697609-b068-4af2-8d7d-c3cc53fb128a  Altınşehir Mahallesi            1

[90 rows x 3 columns]


In [33]:
import pandas as pd

# Orijinal DataFrame: df

# 1. Oturduğu mahalle (en çok bulunduğu mahalle olarak alıyoruz)
mahalle_df = data.groupby('device_aid')['neighborhood'].agg(
    lambda x: x.value_counts().idxmax()).reset_index().rename(columns={'neighborhood': 'oturdugu_mahalle'})

# 2. Restoran türü sayımları
restoran_df = data[data['restaurant_cluster'].notna()]
restoran_pivot = restoran_df.pivot_table(
    index='device_aid',
    columns='restaurant_cluster',
    values='date',
    aggfunc='count',
    fill_value=0
).reset_index()

# Kolon isimlerini düzenleyelim
restoran_pivot.columns.name = None
restoran_pivot = restoran_pivot.rename(columns={
    'Zengin Restoran': 'gidilen_zengin_restoran',
    'Orta Halli Restoran': 'gidilen_orta_restoran',
    'Ucuz Restoran': 'gidilen_ucuz_restoran'
})

# 3. Gidilen lüks villa sayısı
luks_villa_df = data[data['matched_turkey_luxury_houses'].notna()]
villa_counts = luks_villa_df.groupby('device_aid').size().reset_index(name='gidilen_luks_villa')

# 4. Gidilen Türkiye siteleri sayısı
turk_site_df = data[data['matched_turkey_sites'].notna()]
site_counts = turk_site_df.groupby('device_aid').size().reset_index(name='gidilen_turk_site')

# 5. Hepsini birleştirelim
final_df = mahalle_df \
    .merge(restoran_pivot, on='device_aid', how='left') \
    .merge(villa_counts, on='device_aid', how='left') \
    .merge(site_counts, on='device_aid', how='left')

# 6. Boş kalan değerleri 0 ile doldur
final_df[['gidilen_zengin_restoran', 'gidilen_orta_restoran', 'gidilen_ucuz_restoran',
          'gidilen_luks_villa', 'gidilen_turk_site']] = final_df[[
              'gidilen_zengin_restoran', 'gidilen_orta_restoran', 'gidilen_ucuz_restoran',
              'gidilen_luks_villa', 'gidilen_turk_site'
          ]].fillna(0).astype(int)

# Sonuç
print(final_df.head())


                             device_aid     oturdugu_mahalle  \
0  01069bba-c0ed-4999-8515-22754c3fee3e    Atatürk Mahallesi   
1  02712699-4a57-400b-a04a-d39bd5c8eba1  İçerenköy Mahallesi   
2  02d49c46-cfba-4bfb-9a46-5fa417c4ac6e   Barbaros Mahallesi   
3  05e5adce-840a-430c-81d7-22a7ca96f534    Suadiye Mahallesi   
4  0773956e-e6f8-459e-961f-ec1470c80462  Yenişehir Mahallesi   

   gidilen_orta_restoran  gidilen_ucuz_restoran  gidilen_zengin_restoran  \
0                      7                      0                        4   
1                      0                      0                        0   
2                      0                      2                        3   
3                      3                      1                        0   
4                      1                      1                        1   

   gidilen_luks_villa  gidilen_turk_site  
0                   0                  0  
1                   0                  1  
2                   0        

In [34]:
final_df.head()

,device_aid,oturdugu_mahalle,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,7,0,4,0,0
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,0,0,0,0,1
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,0,2,3,0,2
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,3,1,0,1,0
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,1,1,1,0,0


In [35]:
# Ağırlıklar
weights = {
    'gidilen_zengin_restoran': 10,
    'gidilen_orta_restoran': 5,
    'gidilen_ucuz_restoran': 1,
    'gidilen_luks_villa': 12,
    'gidilen_turk_site': 8
}

# Mahalle seviyesi katkısı (sabit değer, ama skora dahil edilecek)
mahalle_score_map = {'A': 25, 'B': 15, 'C': 5}

def calculate_rich_score_v2(row):
    total_score = 0
    total_weight = 0

    # Mahalle seviyesi sabit katkı → direkt puana eklenir, ağırlığa değil
    mahalle = row['mahalle_seviyesi']
    if pd.notna(mahalle):
        total_score += mahalle_score_map[mahalle]

    # Geri kalan katkılar (sayı × ağırlık)
    for col, weight in weights.items():
        val = row[col]
        if pd.notna(val) and val > 0:
            total_score += val * weight
            total_weight += val * weight  # Burada sayı da hesaba giriyor

    # Eğer hiç katkı yoksa rich_score = 0
    if total_weight == 0:
        return 0

    # Normalize et: sadece sayı-ağırlık katkılarına göre 100’lük bazda hesap
    normalized_score = (total_score / (total_weight + 25)) * 100
    return round(normalized_score, 2)


In [36]:
final_df

,device_aid,oturdugu_mahalle,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,7,0,4,0,0
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,0,0,0,0,1
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,0,2,3,0,2
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,3,1,0,1,0
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,1,1,1,0,0
...,...,...,...,...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,1,0,1,0,0
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,5,0,0,0,0
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,9,1,0,0,0
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,0,3,0,0,0


In [37]:
final_df.head()

,device_aid,oturdugu_mahalle,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,7,0,4,0,0
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,0,0,0,0,1
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,0,2,3,0,2
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,3,1,0,1,0
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,1,1,1,0,0


In [38]:
import pandas as pd

def map_mahalle_seviyesi(df, excel_path):
    def process_excel(excel_path):
        df_excel = pd.read_excel(excel_path, sheet_name="Sayfa3", header=None)
        
        # Başlık sütunlarını bulma
        ad_columns = []
        ses_columns = []
        
        for col_idx in range(df_excel.shape[1]):
            cell_value = str(df_excel.iloc[7, col_idx]).lower().strip()
            if "ad" in cell_value and cell_value != "nan":
                ad_columns.append(col_idx)
            if "ses" in cell_value and cell_value != "nan":
                ses_columns.append(col_idx)
        
        # Veri toplama
        mahalle_list = []
        for ad_col, ses_col in zip(ad_columns, ses_columns):
            temp_df = df_excel.iloc[8:, [ad_col, ses_col]].copy()
            temp_df.columns = ["mahalle", "seviye"]
            temp_df = temp_df.dropna(subset=["mahalle", "seviye"])
            mahalle_list.append(temp_df)
        
        full_df = pd.concat(mahalle_list, ignore_index=True)
        
        # Temizlik ve tekilleştirme (ilk kaydı tut)
        full_df["mahalle"] = (
            full_df["mahalle"]
            .str.replace(" Mahallesi", "", case=False)
            .str.strip()
            .str.lower()
        )
        full_df["seviye"] = full_df["seviye"].str.replace(" Grubu", "").str.strip()
        
        full_df = full_df.drop_duplicates(subset=["mahalle"], keep="first")  # Kritik değişiklik
        
        return full_df

    # Eşleştirme işlemleri
    mahalle_seviyeleri = process_excel(excel_path)
    
    df["clean_mahalle"] = (
        df["oturdugu_mahalle"]
        .str.replace(" Mahallesi", "", case=False)
        .str.strip()
        .str.lower()
    )
    
    merged_df = df.merge(
        mahalle_seviyeleri,
        left_on="clean_mahalle",
        right_on="mahalle",
        how="left"
    )
    
    # Son düzenlemeler
    merged_df["mahalle_seviyesi"] = merged_df["seviye"]
    return merged_df.drop(columns=["clean_mahalle", "mahalle", "seviye"])



In [39]:
df_guncel = map_mahalle_seviyesi(final_df, "data/Ilce_Demografi.xlsx")

/tmp/ipykernel_213939/471519981.py:46: PerformanceWarning: Falling back on a non-pyarrow code path which may decrease performance.
  .str.replace(" Mahallesi", "", case=False)


In [40]:
df_guncel

,device_aid,oturdugu_mahalle,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site,mahalle_seviyesi
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,7,0,4,0,0,A
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,0,0,0,0,1,A
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,0,2,3,0,2,A
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,3,1,0,1,0,A
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,1,1,1,0,0,A
...,...,...,...,...,...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,1,0,1,0,0,A
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,5,0,0,0,0,C
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,9,1,0,0,0,C
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,0,3,0,0,0,A


In [41]:
df_guncel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   device_aid               99 non-null     string
 1   oturdugu_mahalle         99 non-null     string
 2   gidilen_orta_restoran    99 non-null     int64 
 3   gidilen_ucuz_restoran    99 non-null     int64 
 4   gidilen_zengin_restoran  99 non-null     int64 
 5   gidilen_luks_villa       99 non-null     int64 
 6   gidilen_turk_site        99 non-null     int64 
 7   mahalle_seviyesi         99 non-null     object
dtypes: int64(5), object(1), string(2)
memory usage: 11.7+ KB


In [42]:
# Sütun sıralamasını yeniden düzenle
cols = df_guncel.columns.tolist()
cols.insert(cols.index('oturdugu_mahalle') + 1, cols.pop(cols.index('mahalle_seviyesi')))
df_guncel = df_guncel[cols]

In [43]:
df_guncel

,device_aid,oturdugu_mahalle,mahalle_seviyesi,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,A,7,0,4,0,0
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,A,0,0,0,0,1
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,A,0,2,3,0,2
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,A,3,1,0,1,0
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,A,1,1,1,0,0
...,...,...,...,...,...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,A,1,0,1,0,0
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,C,5,0,0,0,0
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,C,9,1,0,0,0
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,A,0,3,0,0,0


In [44]:
mahalle_seviye_agirlik = {
    'C': 1,
    'B': 2,
    'A': 3
}

df_guncel['mahalle_seviyesi_encoded'] = df_guncel['mahalle_seviyesi'].map(mahalle_seviye_agirlik)  # Yeni bir sütun oluştur

/tmp/ipykernel_213939/3069156706.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_guncel['mahalle_seviyesi_encoded'] = df_guncel['mahalle_seviyesi'].map(mahalle_seviye_agirlik)  # Yeni bir sütun oluştur


In [45]:
df_guncel

,device_aid,oturdugu_mahalle,mahalle_seviyesi,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site,mahalle_seviyesi_encoded
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,A,7,0,4,0,0,3
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,A,0,0,0,0,1,3
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,A,0,2,3,0,2,3
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,A,3,1,0,1,0,3
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,A,1,1,1,0,0,3
...,...,...,...,...,...,...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,A,1,0,1,0,0,3
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,C,5,0,0,0,0,1
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,C,9,1,0,0,0,1
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,A,0,3,0,0,0,3


In [46]:
from sklearn.preprocessing import StandardScaler

# SADECE sayısal sütunları seç ve kopyala (orijinal DataFrame bozulmasın)
numeric_cols = ['gidilen_orta_restoran', 'gidilen_ucuz_restoran', 
              'gidilen_zengin_restoran', 'gidilen_luks_villa', 'gidilen_turk_site']
df_scaled = df_guncel[numeric_cols].copy()

# Standartlaştırma işlemini sadece bu kopya DataFrame'e uygula
scaler = StandardScaler()
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

In [47]:
df_scaled

,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site
0,1.889798,-0.379931,3.805016,-0.276871,-0.199574
1,-0.424036,-0.379931,-0.475627,-0.276871,0.229944
2,-0.424036,0.795480,2.734855,-0.276871,0.659462
3,0.567607,0.207775,-0.475627,1.335498,-0.199574
4,-0.093488,0.207775,0.594534,-0.276871,-0.199574
...,...,...,...,...,...
94,-0.093488,-0.379931,0.594534,-0.276871,-0.199574
95,1.228703,-0.379931,-0.475627,-0.276871,-0.199574
96,2.550893,0.207775,-0.475627,-0.276871,-0.199574
97,-0.424036,1.383185,-0.475627,-0.276871,-0.199574


In [48]:
# Ağırlıkları kullanarak skor hesapla (scaled değerlerle)
df_guncel['rich_score'] = (
    0.35 * df_scaled['gidilen_luks_villa'] +
    0.25 * df_scaled['gidilen_zengin_restoran'] +
    0.15 * df_scaled['gidilen_turk_site'] +
    0.15 * df_scaled['gidilen_orta_restoran'] +
    0.05 * df_scaled['gidilen_ucuz_restoran'] +
    0.25 * df_guncel['mahalle_seviyesi_encoded']  # Encoded sütunu kullan
)

/tmp/ipykernel_213939/646206264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_guncel['rich_score'] = (


In [49]:
df_guncel

,device_aid,oturdugu_mahalle,mahalle_seviyesi,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site,mahalle_seviyesi_encoded,rich_score
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,A,7,0,4,0,0,3,1.838886
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,A,0,0,0,0,1,3,0.486078
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,A,0,2,3,0,2,3,1.411897
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,A,3,1,0,1,0,3,1.164111
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,A,1,1,1,0,0,3,0.768158
...,...,...,...,...,...,...,...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,A,1,0,1,0,0,3,0.738773
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,C,5,0,0,0,0,1,0.169561
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,C,9,1,0,0,0,1,0.397275
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,A,0,3,0,0,0,3,0.509806


In [50]:
from sklearn.preprocessing import MinMaxScaler
df_guncel['rich_score'] = MinMaxScaler(feature_range=(0, 100)).fit_transform(df_guncel[['rich_score']])

/tmp/ipykernel_213939/4225190188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_guncel['rich_score'] = MinMaxScaler(feature_range=(0, 100)).fit_transform(df_guncel[['rich_score']])


In [51]:
df_guncel.drop(columns=['mahalle_seviyesi_encoded'], inplace=True) 

/tmp/ipykernel_213939/948777633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_guncel.drop(columns=['mahalle_seviyesi_encoded'], inplace=True)


In [52]:
df_guncel

,device_aid,oturdugu_mahalle,mahalle_seviyesi,gidilen_orta_restoran,gidilen_ucuz_restoran,gidilen_zengin_restoran,gidilen_luks_villa,gidilen_turk_site,rich_score
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,A,7,0,4,0,0,42.850287
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,A,0,0,0,0,1,12.614979
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,A,0,2,3,0,2,33.307062
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,A,3,1,0,1,0,27.769045
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,A,1,1,1,0,0,18.919474
...,...,...,...,...,...,...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,A,1,0,1,0,0,18.262712
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,C,5,0,0,0,0,5.540814
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,C,9,1,0,0,0,10.630227
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,A,0,3,0,0,0,13.145302


In [53]:
df_last = df_guncel[['device_aid', 'oturdugu_mahalle', 'rich_score']]
df_last

,device_aid,oturdugu_mahalle,rich_score
0,01069bba-c0ed-4999-8515-22754c3fee3e,Atatürk Mahallesi,42.850287
1,02712699-4a57-400b-a04a-d39bd5c8eba1,İçerenköy Mahallesi,12.614979
2,02d49c46-cfba-4bfb-9a46-5fa417c4ac6e,Barbaros Mahallesi,33.307062
3,05e5adce-840a-430c-81d7-22a7ca96f534,Suadiye Mahallesi,27.769045
4,0773956e-e6f8-459e-961f-ec1470c80462,Yenişehir Mahallesi,18.919474
...,...,...,...
94,efe3e217-cfb2-4eb9-ad1d-c2680f7fbd91,Fenerbahçe Mahallesi,18.262712
95,f2102cd7-f776-4adf-a782-b333a2ca8cb6,Tepeüstü Mahallesi,5.540814
96,f3717295-5995-482e-ad4a-58157d2ab66c,Tepeüstü Mahallesi,10.630227
97,f99030ff-e47c-446a-b0ce-8dc9651a05ca,Dudullu OSB Mahallesi,13.145302


In [54]:
df_last.to_csv('id-mahalle-rich_score.csv', index=False)